In [4]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

def load_feedback():
    try:
        with open("feedback.json", 'r') as feedback_file:
            feedback_data = json.load(feedback_file)
            return feedback_data.get("upvotes", {}), feedback_data.get("downvotes", {})
    except FileNotFoundError:
        return {}, {}

def save_feedback(upvotes, downvotes):
    feedback_data = {"upvotes": upvotes, "downvotes": downvotes}
    with open("feedback.json", 'w') as feedback_file:
        json.dump(feedback_data, feedback_file, indent=4)

def get_response(intents, tag, upvotes, downvotes):
    for intent in intents['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']
            upvote_count = upvotes.get(tag, 0)
            downvote_count = downvotes.get(tag, 0)

            if upvote_count >= downvote_count:
                return random.choice(intent['responses'])
            else:
                return "I'm sorry, I don't understand that."

    return "I'm sorry, I don't understand that."

def chat_with_ml():
    intents = json.load(open("intents.json", 'r'))
    
    parameters = {'svc__C': [0.1, 1, 10, 100]}  
    model = make_pipeline(TfidfVectorizer(), SVC())

    grid_search = GridSearchCV(model, parameters, cv=3)
    
    patterns = [pattern.lower() for intent in intents['intents'] for pattern in intent['patterns']]
    tags = [intent['tag'] for intent in intents['intents'] for _ in intent['patterns']]
    
    grid_search.fit(patterns, tags)

    initial_upvotes = {}
    initial_downvotes = {}

    upvotes, downvotes = load_feedback()

    for tag in initial_upvotes:
        upvotes[tag] = upvotes.get(tag, 0) + initial_upvotes[tag]

    for tag in initial_downvotes:
        downvotes[tag] = downvotes.get(tag, 0) + initial_downvotes[tag]

    print("Hello! I'm your chatbot Kenal. Type 'exit' to end the conversation.")

    while True:
        user_input = input("You: ").lower()

        if user_input == 'exit':
            print("Goodbye! Have a great day.")
            break

        user_intent = grid_search.predict([user_input])[0]

        print("Kenel:")
        response = get_response(intents, user_intent, upvotes, downvotes)
        print(response)
        
        
        while True:
            feedback = input("Did you find the response helpful? (y/n): ").lower()
            if feedback == 'y':
                upvotes[user_intent] = upvotes.get(user_intent, 0) + 1
                break
            elif feedback == 'n':
                downvotes[user_intent] = downvotes.get(user_intent, 0) + 1
                break
            else:
                print("Invalid input. Please enter 'y' or 'n'.")

        
       

    save_feedback(upvotes, downvotes)

if __name__ == "__main__":
    chat_with_ml()


Hello! I'm your chatbot Kenal. Type 'exit' to end the conversation.
You: hi
Kenel:
Hello!
Did you find the response helpful? (y/n): y
You: who is your creater
Kenel:
Biswajit Sahoo is my creater
Did you find the response helpful? (y/n): y
You: goodbye
Kenel:
Sad to see you go :(
Did you find the response helpful? (y/n): y
You: exit
Goodbye! Have a great day.
